# Implementation of Markov Partition Construction Algorithm in Chapter 4.2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

## Define System's Dynamics in Euclidean Plane and on Torus

In [ ]:
A = np.array([[1,1], [1,0]])

In [ ]:
def phi(x: np.array) -> np.array:
    x_new = np.dot(A, x) % 1
    return x_new

## Compute Eigendecomposition

In [ ]:
# get eigenvalues and right eigenvectors, i.e. transposed eigenvectors
eig_vals, eig_vects = np.linalg.eig(A)
# retrieve eigenvectors from right eigenvectors
eig_vects = np.transpose(eig_vects)

In [ ]:
origin = [0], [0]
plt.quiver(*origin, eig_vects[:,0], eig_vects[:,1], color=['r','b'], scale=3)
plt.show()

In [ ]:
print(f"Eigenvalues: {eig_vals}")

In [ ]:
print(f"Eigenvectors: {eig_vects}")

## Analysis of Hyperbolicity and Invertibility

In [ ]:
def is_hyperbolic(eig_vals: np.array) -> bool:
    contr_seg = any(np.abs(eig_val) < 1 for eig_val in eig_vals)
    exp_seg = any(np.abs(eig_val) > 1 for eig_val in eig_vals)

    return contr_seg and exp_seg

In [ ]:
system_hyperbolic = is_hyperbolic(eig_vals)
print(f"Dynamic system is hyperbolic: {system_hyperbolic}")

In [ ]:
def is_invertible(A: np.array) -> bool:
    return np.abs(np.linalg.det(A)) == 1

In [ ]:
system_invertible = is_invertible(A)
print(f"Dynamic system is invertble: {system_invertible}")

## Computation of Bases for Contracting and Expanding Segments 

In [ ]:
def expanding_segments(eig_vals: np.array, eig_vects: np.array) -> np.array:
    exp_eig_spaces_msk = np.abs(eig_vals) > 1
    print(exp_eig_spaces_msk)
    return eig_vects[exp_eig_spaces_msk]

In [ ]:
def contracting_segments(eig_vals: np.array, eig_vects: np.array) -> np.array:
    contr_eig_spaces_msk = np.abs(eig_vals) < 1
    print(contr_eig_spaces_msk)
    return eig_vects[contr_eig_spaces_msk]

In [ ]:
contr_segs = contracting_segments(eig_vals, eig_vects)
print(f"Contracting segments: {contr_segs}")

In [ ]:
exp_segs = expanding_segments(eig_vals, eig_vects)
print(f"Expanding segments: {exp_segs}")

## Translate between Euclidea Plane and Torus

In [ ]:
def q(x: np.array) -> np.array:
    return x % 1

In [ ]:
# sample point in R^2 between -10 and 10
x = np.random.uniform(low=-10, high=10, size=2)

In [ ]:
# apply system's dynamics A in plane, then push down to torus by q
plane_dynamics_result = q(np.dot(A, x))
print(f"Next point on torus: {plane_dynamics_result}")

In [ ]:
# push down point to torus, then apply system's dynamics phi on torus
torus_dynamics_result = phi(q(x))
print(f"Next point on torus: {torus_dynamics_result}")

In [ ]:
if np.all(plane_dynamics_result == torus_dynamics_result):
    print(f"System dynamics in plane and on torus commute by q!")
else:
    print(f"System dynamics in plane and on torus do not commute by q!")

## Calculate Inetrsections between Lines

In [ ]:
def perp(a) :
    b = np.empty_like(a)
    b[0] = -a[1]
    b[1] = a[0]
    return b

# line segment a given by endpoints a1, a2
# line segment b given by endpoints b1, b2
def seg_intersect(a, b) :
    a1, a2 = a
    b1, b2 = b

    da = a2 - a1
    db = b2 - b1
    dp = a1 - b1
    dap = perp(da)
    denom = np.dot(dap, db)
    num = np.dot(dap, dp)

    return (num / denom.astype(float))*db + b1

## Calculate Hyperbolic Fixed Point

In [ ]:
# reformulate fixed point problem Ax = x as root finding problem Ax-x = 0
def objective_func(x: np.array) -> np.array:
    return np.dot(A, x) - x

In [ ]:
# function that calculates jacobian of objective function, namely D(Ax-x) = A-I
def jac(x: np.array) -> np.array:
    return A - np.identity(n=2)

In [ ]:
# find roots of objective function with Levenberg-Marquardt algorithm
root_sol = optimize.root(objective_func, np.array([1,2]), jac=jac, method="lm")
if not root_sol.success:
    print(f"Failed to find a fixed point. Error: {root_sol.message}")
else:
    fixed_point = root_sol.x
    print(f"Fixed point: {fixed_point}")